## BlazingSQL Logs

[Docs](https://docs.blazingdb.com/docs/blazingsql-logs) 

BlazingSQL has an internal log that records events from every node from all queries run. The events include runtime query step execution information, performance timings, errors and warnings. 

The logs table is called `bsql_logs`. You can query the logs as if it were any other table, except you use the `.log()` function, instead of the `.sql()` function.

In [1]:
from blazingsql import BlazingContext
bc = BlazingContext()

BlazingContext ready


In [2]:
bc.log('select * from bsql_logs').head()

,log_time,node_id,type,query_id,step,substep,info,duration,extra1,data1,extra2,data2
0,1969-12-31 23:59:59.999,null,None,null,null,null,None,null,None,null,None,null
1,2020-03-11 20:13:31.000,0,TRACE,0,0,0,"INITIALIZING RAL. RAL ID: 0, RAL Host: 127.0.0...",null,None,null,None,null
2,1976-01-20 05:00:48.000,null,None,null,null,null,None,null,None,null,None,null
3,2020-03-12 19:02:31.000,0,TRACE,0,0,0,"INITIALIZING RAL. RAL ID: 0, RAL Host: 127.0.0...",null,None,null,None,null
4,1969-12-31 23:59:59.999,null,None,null,null,null,None,null,None,null,None,null


How long did each successfully run query take?

In [3]:
bc.log("SELECT log_time, query_id, duration FROM bsql_logs WHERE info = 'Query Execution Done' ORDER BY log_time DESC")

,log_time,query_id,duration
0,2020-03-12 19:36:14,16654,26.158203
1,2020-03-12 19:36:01,49237,52.651920
2,2020-03-12 19:30:09,46726,26.591824
3,2020-03-12 19:27:56,15725,23.990410
4,2020-03-12 19:27:56,59508,21.562897
5,2020-03-12 19:27:26,49708,16.824747
6,2020-03-12 19:27:01,12268,20.589977
7,2020-03-12 19:26:31,61114,16.944666
8,2020-03-12 19:13:45,17253,8.537910
9,2020-03-12 19:10:55,48164,7.966240


This query determines the data load time and total time for all queries, showing the latest ones first. 

Load time and total time being the maximum load time and total time for any node.

In [4]:
log_query = '''
            SELECT
                MAX(end_time) as end_time, query_id, 
                MAX(load_time) AS load_time, MAX(total_time) AS total_time 
            FROM (
                SELECT
                    query_id, 
                    node_id,
                    SUM(CASE WHEN info = 'evaluate_split_query load_data' THEN duration ELSE 0 END) AS load_time,
                    SUM(CASE WHEN info = 'Query Execution Done' THEN duration ELSE 0 END) AS total_time,
                    MAX(log_time) AS end_time
                FROM
                    bsql_logs
                    WHERE
                        info = 'evaluate_split_query load_data'
                        OR info = 'Query Execution Done'
                GROUP BY
                    node_id, query_id
                    )
            GROUP BY
                query_id
            ORDER BY
                end_time DESC
                '''
bc.log(log_query)

,end_time,query_id,load_time,total_time
0,2020-03-12 19:36:15,15800,16.922256,21.231842
1,2020-03-12 19:36:14,16654,22.460842,26.158203
2,2020-03-12 19:36:01,49237,30.908297,52.651920
3,2020-03-12 19:30:09,46726,17.911966,26.591824
4,2020-03-12 19:27:56,59508,15.452898,21.562897
5,2020-03-12 19:27:56,15725,20.697012,23.990410
6,2020-03-12 19:27:26,49708,11.722969,16.824747
7,2020-03-12 19:27:01,12268,16.528986,20.589977
8,2020-03-12 19:26:31,61114,13.743263,16.944666
9,2020-03-12 19:13:45,17253,7.577676,8.537910


If you run the same queries multiple times, this query against the logs will tell you the average execution time for every query.

In [5]:
query = """
        SELECT 
            MAX(end_time) AS end_time, 
            SUM(query_duration)/COUNT(query_duration) AS avg_time,
            MIN(query_duration) AS min_time, 
            MAX(query_duration) AS max_time, 
            COUNT(query_duration) AS num_times, 
            relational_algebra 
        FROM (
            SELECT
                times.end_time as end_time, 
                times.query_id, times.avg_time,
                times.max_time as query_duration, 
                times.min_time, 
                ral.relational_algebra as relational_algebra
            FROM (
                SELECT
                    query_id, 
                    MAX(log_time) AS end_time, 
                    SUM(duration)/COUNT(duration) AS avg_time,
                    MIN(duration) AS min_time, 
                    MAX(duration) AS max_time
                FROM
                    bsql_logs
                    WHERE
                        info = 'Query Execution Done'
                    GROUP BY
                        query_id 
                        ) 
                        AS times
            INNER JOIN (
                SELECT
                    query_id, 
                    SUBSTRING(info, 13, 2000) AS relational_algebra
                FROM
                    bsql_logs
                    WHERE
                        info LIKE 'Query Start%'
                GROUP BY
                    query_id, info 
                    ) 
                    AS ral
                ON
                    times.query_id = ral.query_id
            ORDER BY
                times.end_time DESC) 
                AS temp 
            GROUP BY 
                relational_algebra 
                """
bc.log(query)

,end_time,avg_time,min_time,max_time,num_times,relational_algebra
0,2020-03-12 19:13:45,8.252075,7.966240,8.537910,2,"LogicalTableScan(table=[[main, karate]])\n"
1,2020-03-12 19:36:14,21.920813,16.944666,26.158203,4,"LogicalTableScan(table=[[main, bsql_logs]])\n"
2,2020-03-12 19:36:15,25.977158,25.362492,26.591824,2,"LogicalSort(sort0=[$0], dir0=[DESC])\n Logica..."
3,2020-03-12 19:36:01,52.651920,52.651920,52.651920,1,"LogicalProject(end_time=[$1], avg_time=[/($2, ..."
4,2020-03-12 19:36:15,19.873161,16.824747,21.562897,3,"LogicalSort(sort0=[$0], dir0=[DESC])\n Logica..."
